In [1]:
import importlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import surprise

import hw5_version
import hw5bias_version_g2
import surprise_version
importlib.reload(hw5_version)
importlib.reload(hw5bias_version_g2)
importlib.reload(surprise_version)

%config InlineBackend.figure_format = 'retina'


In [2]:
def visualize_V(U, V, labels=None, index=None, name='temp', save=True, title=None, hue=None,size=None):
    A, s, B = np.linalg.svd(V.T)
    U_proj = A[:,0:2].T@U.T
    V_proj = A[:,0:2].T@V.T
    plt.figure(figsize=(8,4))
    
    
    if index is None:
        plot = sns.scatterplot(V_proj[0,:],V_proj[1,:],hue=hue,size=size,legend=False)
    else : 
        plot = sns.scatterplot(V_proj[0,index-1],V_proj[1,index-1],hue=hue[index],size=size[index],legend=False)
        
#     plot._legend.remove()
        
    limits = plt.ylim()
    offset = (limits[1]-limits[0])*0.02
    if not (labels is None):
        for i, label in enumerate(labels):
            txt = plot.text(V_proj[0,index[i]-1],V_proj[1,index[i]-1]+offset, label, horizontalalignment='center')
            relative_location = (V_proj[0,index[i]-1]-plt.xlim()[0])/(plt.xlim()[1] - plt.xlim()[0]) 
            if relative_location < 0.2:
                txt.set(horizontalalignment='left')
            elif relative_location > 0.8:
                txt.set(horizontalalignment='right')
            
    plt.title(title)
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    if save:
        fig = plot.get_figure()
        fig.savefig(f'{name}.png')
        fig.savefig(f'{name}.pdf')
    

In [3]:
def multi_vis(U, V, name='temp',save=True,hue=None,size=None):
    df = pd.read_csv('./data/movies3.txt',sep='\t',header=None)
    movies = df[1]
    
    # Ten Movies of our choice
    choice = np.arange(1,11)
    visualize_V(U,V,index=choice,name=f'{name}_choice',labels=movies[choice-1],save=save,title=f'Arbitrarily Chosen Movies with {name}',hue=hue,size=size)
    
    # Ten Most Popular
#     popular = np.array([50, 258, 100, 181, 294, 286, 288, 1, 300, 121])
    popular = np.array([50, 258, 100, 181, 293, 285, 287, 1, 299, 121])
    visualize_V(U,V,index=popular,name=f'{name}_popular',labels=movies[popular-1],save=save,title=f'Most Popular Movies with {name}',hue=hue,size=size)
    
    # Ten Best
#     best = np.array([1189, 1500, 814, 1536, 1293, 1599, 1653, 1467, 1122, 1201])
    best = np.array([1180, 1487, 808, 1523, 1283, 1581, 1628, 1454, 1113, 1192])
    visualize_V(U,V,index=best,name=f'{name}_best',labels=movies[best-1],save=save,title=f'\'Best\' Movies with {name}',hue=hue,size=size)
    
    # Ten from each of three genres
#     animation = np.array([1, 71, 95, 99, 101, 102, 103, 114, 169, 189])
    animation = np.array([1, 71, 95, 99, 101, 102, 103, 114, 169, 189])
#     documentaries = np.array([32, 48, 75, 115, 119, 320, 360, 634, 644, 645])
    documentaries = np.array([32, 48, 75, 115, 119, 318, 357, 630, 640, 641])
#     sci_fi = np.array([7, 38, 39, 50, 62, 82, 84, 89, 96, 101])
    sci_fi = np.array([7, 38, 39, 50, 62, 82, 84, 89, 96, 101])
                       
    # Genre 1
    visualize_V(U,V,index=animation,name=f'{name}_animation',labels=movies[animation-1],save=save,title=f'Animation Movies with {name}',hue=hue,size=size)
    # Genre 2
    visualize_V(U,V,index=documentaries,name=f'{name}_documentary',labels=movies[documentaries-1],save=save,title=f'Documentary Movies with {name}',hue=hue,size=size)
    # Genre 3
    visualize_V(U,V,index=sci_fi,name=f'{name}_scifi',labels=movies[sci_fi-1],save=save,title=f'SciFi Movies with {name}',hue=hue,size=size)

In [4]:
Y_train = np.loadtxt('./data/train3.txt').astype(int)
Y_test = np.loadtxt('./data/test3.txt').astype(int)

M = max(max(Y_train[:,0]), max(Y_test[:,0])).astype(int) # users
N = max(max(Y_train[:,1]), max(Y_test[:,1])).astype(int) # movies


K = 20
eta = 0.03
reg = 0
reg = 0.1


# Using the Homework 5 version of SVD 

In [5]:
U,V, e_in_base = hw5_version.train_model(M, N, K, eta, reg, Y_train)

In [6]:
visualize_V(U,V, hue=hue,size=size)

NameError: name 'hue' is not defined

In [ ]:
print(e_in_base)
e_out_base = hw5_version.get_err(U,V,Y_test)
print(hw5_version.get_err(U,V,Y_test))

# Using the implementation with the additional bias terms

In [ ]:
U_bias, V_bias, a_bias, b_bias, mu_bias, e_in_bias = hw5bias_version_g2.train_model(M, N, K, eta, reg, Y_train)

In [ ]:
print(e_in_bias)
e_out_bias = hw5bias_version_g2.get_err(U_bias,V_bias, Y_test, a_bias, b_bias, mu_bias)
print(e_out_bias)

In [ ]:
visualize_V(U_bias,V_bias)

# Using the off the shelf implementation of SVD.

In [ ]:
model,  e_in_sur, trainset = surprise_version.train_model(M, N, K, eta, reg, Y_train)

In [ ]:
visualize_V(model.pu,model.qi)

In [ ]:
print(e_in_sur)
e_out_sur = surprise_version.get_err(model,Y_test)
print(surprise_version.get_err(model,Y_test))

# Create the visualizations

In [ ]:
hue = np.asarray(list(map(lambda movie_id : np.mean(Y_train[Y_train[:,1]==movie_id,2]), np.arange(N)+1)))
size = np.asarray(list(map(lambda movie_id : np.sum(Y_train[:,1]==movie_id), np.arange(N)+1)))
multi_vis(U,V,name='Basic Factorization',hue=hue,size=size)
multi_vis(U_bias,V_bias,name='Factorization with Bias',hue=hue,size=size)
multi_vis(model.pu,model.qi,name='Surprise Factorization',hue=hue[],size=size[])

# Visualize all movies for each method

In [ ]:
visualize_V(U,V,name='Basic Factorization',hue=hue,size=size)
visualize_V(U_bias,V_bias,name='Factorization with Bias',hue=hue,size=size)
print(model.qi.shape)
print(hue.shape)
visualize_V(model.pu,model.qi,name='Surprise Factorization',hue=hue,size=size)

# Plot the performance

In [ ]:

temp = np.array([[e_in_base,e_out_base],[e_in_bias,e_out_bias],[e_in_sur,e_out_sur]])
for i in range(2):
    plt.bar(x=np.arange(0,3)*3+i-0.5,height=temp[:,i])
    plt.legend(['$E_{in}$','$E_{out}$'])
    plt.title('Performance of Various Factorization Techniques')
    plt.ylabel('Error')
    plt.xticks(np.arange(0,3)*3, ['Basic Factorization', 'Factorization With Bias', 'Surprise Factorization'])
    
name = 'performace'
plt.savefig(f'{name}.png')
plt.savefig(f'{name}.pdf')